In [ ]:
# import the datafiles from the output folder of pt2 and pt3

from tkinter.filedialog import askdirectory
import os
import pandas as pd

# Select output folder from pt2
ratios_folder = askdirectory(title='Select the output folder from pt2')
hics_folder = askdirectory(title='Select the output folder from pt3')

# ratios_folder = 'C:/Users/BIO/Desktop/Clyde Local/Calculating HIC with peak acceleration and pulse duration (IRCOBI24, AAAM24)/2-3. Head acceleration pulse analysis/output/Data extracted on Apr 08, 2024 at 04 22 PM_final'
# hics_folder = 'C:/Users/BIO/Desktop/Clyde Local/Calculating HIC with peak acceleration and pulse duration (IRCOBI24, AAAM24)/2-3. Head acceleration pulse analysis/output/HIC output calculated on Apr 09, 2024 at 08 11 AM_final'

ascii_folder = f'{ratios_folder}/ASCII files/'

ascii_files = [f for f in os.listdir(ascii_folder) if f.endswith('.txt')]

print(f"Found {len(ascii_files)} test datasets.")

ratio = pd.read_excel(f'{ratios_folder}/Combined width output.xlsx')
train_ratios = ratio.query("Type == 'Train'")
test_ratios = ratio.query("Type == 'Test'")

print(f"Found {len(train_ratios)} training datasets and {len(test_ratios)} test datasets.")

results = pd.read_excel(f'{hics_folder}/HIC comparison output.xlsx')

# Check for outliers

In [ ]:
# remove rows  with HIC values greater than 3x the standard deviation for Full Pulse HIC	Haversine HIC15	Sine HIC15	Quadratic HIC15	Triangular HIC15	Rectangular HIC15
hics = results
cols = ['Full Pulse HIC', 'Haversine HIC15', 'Sine HIC15', 'Quadratic HIC15', 'Triangular HIC15', 'Rectangular HIC15']
averages = [hics[col].mean() for col in cols]
sds = [hics[col].std() * 3 for col in cols]
for idx, col in enumerate(cols):
    before_length = len(hics)
    hics = hics.query(f"`{col}` <= {averages[idx] + sds[idx]} and `{col}` >= {averages[idx] - sds[idx]}")
    print(f"{before_length - len(hics)} outliers removed from {col}")

print(f"final length of results: {len(hics)}")

# Figure 1: example plot
Annotated plot with overlaid t<sub>i, t<sub>f, t<sub>x,max and t<sub>belt values.

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.kaleido.scope.chromium_args += ("--single-process",)

test_number = 11584
height_inches = 2
width_inches = 3
font_size = 10
dpi = 300
filename = 'Figure_1'

file_name = f'{ascii_folder}/{test_number} 0 True.txt'
head_acceleration = pd.read_csv(file_name, sep='\t', skiprows=1, names=['Time', 'Acceleration'])
head_acceleration['Time'] = head_acceleration['Time'] * 1000
test_statistics = train_ratios.query(f"`Test Number` == {test_number}")
timepoints = test_statistics[['ti', 'tf', 'tpeak', 'txmax', 'tbelt']].values[0]

fig = px.line(head_acceleration, x='Time', y='Acceleration', title='', width=width_inches * dpi, height=height_inches * dpi, template='simple_white')
fig.update_layout(xaxis_title='Time (ms)',
                yaxis_title='Resultant Head Acceleration (g)',
                showlegend=False,
                font={'size': round(font_size * (dpi / 96)), 'family': "Times New Roman"},
                xaxis_range=[0, 300], yaxis_range=[0, head_acceleration.Acceleration.max() * 1.2],
                margin={'l': 20, 'r': 20, 't': 10, 'b': 20})

# timepoint_labels = ['t<sub>i', 't<sub>f', 'P<sub>exp', 't<sub>xmax', 't<sub>belt']
# for idx, timepoint in enumerate(timepoints):
#     offset = 1.1
#     fig.add_scatter(x=[timepoint, timepoint], y=[0, head_acceleration.Acceleration.max() * offset], mode='lines+text', text=["", f"{timepoint_labels[idx]}"], textposition="top center",
#                         line=dict(dash='dash', color='black'))
fig.add_scatter(x=[timepoints[0], timepoints[0]], y=[0, head_acceleration.Acceleration.max() * 1.1], mode='lines+text', text=["", "t<sub>i"], textposition="top center",
                        line=dict(dash='dash', color='black'))
fig.add_scatter(x=[timepoints[1], timepoints[1]], y=[0, head_acceleration.Acceleration.max() * 1.1], mode='lines+text', text=["", "t<sub>f"], textposition="top center",
                        line=dict(dash='dash', color='black'))
fig.add_scatter(x=[timepoints[2], timepoints[2]], y=[0, head_acceleration.Acceleration.max() * 1.1], mode='lines+text', text=["", "t<sub>P"], textposition="top center",
                        line=dict(dash='dash', color='black'))
fig.add_scatter(x=[timepoints[3], timepoints[3]], y=[0, head_acceleration.Acceleration.max() * 1], mode='lines+text', text=["", "t<sub>xmax"], textposition="top center",
                        line=dict(dash='dash', color='black'))
fig.add_scatter(x=[timepoints[4], timepoints[4]], y=[0, head_acceleration.Acceleration.max() * 1], mode='lines+text', text=["", "t<sub>belt"], textposition="top center",
                        line=dict(dash='dash', color='black'))

figure_name = f"{hics_folder}/{filename}"
fig.write_image(file=f"{figure_name}.png", format = "png", width=width_inches * dpi, height=height_inches * dpi, scale=1, engine='kaleido')
os.startfile(f"{figure_name}.png")

# Figure A1: example plot with peak and duration

In [ ]:
test_number = 11064
height_inches = 3.21
width_inches = 4.29
font_size = 16
dpi = 300
filename='Figure_A1'

file_name = f'{ascii_folder}/{test_number} 0 True.txt'
head_acceleration = pd.read_csv(file_name, sep='\t', skiprows=1, names=['Time', 'Acceleration'])
head_acceleration['Time'] = head_acceleration['Time'] * 1000
test_statistics = train_ratios.query(f"`Test Number` == {test_number}")

fig = px.line(head_acceleration, x='Time', y='Acceleration', title='', width=width_inches * dpi, height=height_inches * dpi, template='simple_white')
fig.update_layout(xaxis_title='Time (ms)',
                yaxis_title='Resultant Head Acceleration (g)',
                showlegend=False,
                font={'size': round(font_size * (dpi / 96)), 'family': "Times New Roman"},
                xaxis_range=[0, 200], yaxis_range=[0, head_acceleration.Acceleration.max() * 1.2],
                margin={'l': 20, 'r': 20, 't': 10, 'b': 20})

t_peak = test_statistics['tpeak'].values[0]
fig.add_scatter(x=[0, t_peak], y=[head_acceleration.Acceleration.max(), head_acceleration.Acceleration.max()], mode='lines', line=dict(dash='dash', color='black'))
fig.add_scatter(x=[t_peak], y=[head_acceleration.Acceleration.max() * 1.02], mode='text', text=['P<sub>exp'], textposition='top center')

ti = test_statistics['ti'].values[0]
tf = test_statistics['tf'].values[0]

# fig.add_scatter(x=[ti, ti], y=[0, head_acceleration.Acceleration.max() * 1.1], mode='lines', textposition="top center",
#                     line=dict(dash='dash', color='black'))
# fig.add_scatter(x=[tf, tf], y=[0, head_acceleration.Acceleration.max() * 1.1], mode='lines', textposition="top center",
#                     line=dict(dash='dash', color='black'))

fig.add_annotation(x=ti, y=2, ax=tf, ay=2, xref='x', yref='y', axref = 'x', ayref = 'y', text="", showarrow=True, arrowhead=5, arrowwidth=3)
fig.add_annotation(x=tf, y=2, ax=ti, ay=2, xref='x', yref='y', axref = 'x', ayref = 'y', text="", showarrow=True, arrowhead=5, arrowwidth=3)
fig.add_scatter(x=[t_peak], y=[3], mode='text', text=['d<sub>exp'], textposition='top center')

# timepoint_labels = ['t<sub>peak', 't<sub>xmax', 't<sub>belt']
# for idx, timepoint in enumerate(timepoints):
#     offset = 1.1
#     fig.add_scatter(x=[timepoint, timepoint], y=[0, head_acceleration.Acceleration.max() * offset], mode='lines+text', text=["", f"{timepoint_labels[idx]}"], textposition="top center",
#                         line=dict(dash='dash', color='black'))

figure_name = f"{hics_folder}/{filename}"
fig.write_image(file=f"{figure_name}.png", format = "png", width=width_inches * dpi, height=height_inches * dpi, scale=1, engine='kaleido')
os.startfile(f"{figure_name}.png")

# Figure A3: hic boxplot

In [ ]:
font_size = 10
dpi = 300
width_inches = 6.5
height_inches = 3

plot_hics = hics[['Full Pulse HIC', 'Rectangular HIC15', 'Triangular HIC15', 'Quadratic HIC15', 'Sine HIC15', 'Haversine HIC15']]
# plot_hics = plot_hics.set_axis(['Experimental', 'Haversine', 'Sine', 'Quadratic', 'Triangular', 'Rectangular'], axis='columns')
plot_hics = plot_hics.set_axis(['Experimental', 'Rectangular', 'Triangular', 'Quadratic', 'Sine', 'Haversine'], axis='columns')
fig = px.box(plot_hics, title='', template='simple_white', points='all')
fig.update_layout(yaxis_title='HIC<sub>15',
                  xaxis_title = 'Trace used for HIC<sub>15</sub> calculation',
                  font={'size': round(font_size * (dpi / 96)), 'family': "Times New Roman"},
                  margin={'l': 20, 'r': 20, 't': 10, 'b': 20})

fig.show()
figure_name = f"{hics_folder}/HIC Comparison"
fig.write_image(file=f"{figure_name}.png", format = "png", width=width_inches * dpi, height=height_inches * dpi, scale=1, engine='kaleido')
os.startfile(f"{figure_name}.png")

# Figure: artistic scatterplot

In [ ]:
# font_size = 10
# dpi = 300
# filename='Figure_A3'
# width_inches = 7.5
# height_inches = 5

# timepoints = train_ratios[['ti', 'tpeak', 'txmax', 'tf', 'tbelt']]
# timepoints = pd.melt(timepoints)
# values = train_ratios[['ati', 'atpeak', 'atxmax', 'atf', 'atbelt']]
# values = pd.melt(values)

# points = pd.concat([timepoints, values['value']], axis=1).set_axis(['var', 'time', 'amp'], axis=1)

# labels = {'ti': 't<sub>i', 'tf': 't<sub>f', 'tpeak': 't<sub>peak', 'txmax': 't<sub>xmax', 'tbelt': 't<sub>belt'}
# points['var'] = points['var'].apply(lambda x: labels.get(x))

# symbols_list = ['circle', 'square', 'diamond', 'cross', 'x'] * len(points)

# fig = px.scatter(points, x='time', y='amp', title='', marginal_y='box', marginal_x='box', symbol='var', template='simple_white', color='var')
# fig.update_layout(xaxis_title='Time (ms)',
#                   yaxis_title='Acceleration (g)',
#                   legend=dict(y=0.82, x=0.01, title='', borderwidth=2),
#                   font={'size': round(font_size * (dpi / 96)), 'family': "Times New Roman"},
#                   margin={'l': 20, 'r': 20, 't': 10, 'b': 20})
# fig.update_xaxes(mirror=True)
# fig.update_yaxes(mirror=True)
# fig.update_traces(marker=dict(size=15))

# fig.layout.yaxis2.mirror = False
# fig.layout.xaxis3.mirror = False

# fig.layout.yaxis2.visible = False
# fig.layout.xaxis3.visible = False

# fig.layout.yaxis3.categoryorder = "array"
# fig.layout.yaxis3.categoryarray = ['t<sub>belt', 't<sub>f', 't<sub>peak', 't<sub>xmax', 't<sub>i']

# fig.layout.xaxis2.showticklabels = True
# fig.layout.xaxis2.side = "bottom"
# fig.layout.yaxis3.showticklabels = True
# fig.layout.yaxis3.side = "left"

# fig.layout.xaxis.domain = [0.0, 0.83]
# fig.layout.xaxis2.domain = [0.83, 1.0]
# fig.layout.yaxis.domain = [0.0, 0.83]
# fig.layout.yaxis3.domain = [0.83, 1.0]

# figure_name = f"{hics_folder}/{filename}"
# fig.write_image(file=f"{figure_name}.png", format = "png", width=width_inches * dpi, height=height_inches * dpi, scale=1, engine='kaleido')
# os.startfile(f"{figure_name}.png")

# pd.set_option('display.max_rows', 1000)

# Figure A4: Overlaid modeled curves

In [ ]:
import numpy as np
test_number = 5326
height_inches = 2
width_inches = 3
font_size = 10
dpi = 300
filename='Figure_A4'

file_name = f'{ascii_folder}/{test_number} 0 True.txt'
head_acceleration = pd.read_csv(file_name, sep='\t', skiprows=1, names=['Time', 'Acceleration'])
head_acceleration['Time'] = head_acceleration['Time'] * 1000
test_statistics = train_ratios.query(f"`Test Number` == {test_number}")

def fit_curves(peak_acceleration: float, pulse_width: float, fs: float) -> pd.DataFrame:
        # define time array
        pulse_time = np.linspace(0, pulse_width, round(pulse_width * fs))

        # Haversine
        pulse_haversine = peak_acceleration * (np.sin(np.pi * pulse_time / pulse_width))**2

        # Sine
        pulse_sine = peak_acceleration * np.sin(np.pi * pulse_time / pulse_width)

        # Triangular
        triangular = peak_acceleration * (1 - abs(((2 * pulse_time) / pulse_width)-1))

        # Rectangular
        rectangular = np.repeat(peak_acceleration, len(pulse_time))

        # Quadratic
        quadradtic = -4 * peak_acceleration / (pulse_width**2) * (pulse_time - pulse_width / 2)**2 + peak_acceleration

        curve_names = ['Haversine', 'Sine', 'Triangular', 'Rectangular', 'Quadratic']
        curves = [pulse_haversine, pulse_sine, triangular, rectangular, quadradtic]

        return pd.DataFrame([curve_names, curves], index=['Curve', 'Data']).T

display(test_statistics['Peak Acceleration (g)'].values[0])
peak_acceleration = test_statistics['Peak Acceleration (g)'].values[0]
peak_time = test_statistics['tpeak'].values[0]

trunc_accel = head_acceleration.query('(Time >= 0) and (Time <= 200)')
full_pw = test_statistics['Full Pulse Width (ms)'].values[0]

ti = test_statistics['ti'].values[0]
tf = test_statistics['tf'].values[0]
fs = 1 / 1000
duration = full_pw * 0.1173
h_duration  = duration
rect_time = head_acceleration.query(f'(Time >= {peak_time - h_duration}) and (Time <= {peak_time + h_duration})')['Time']
trunc_accel['rect'] = 0
trunc_accel.loc[head_acceleration.Time.between(peak_time - h_duration, peak_time + h_duration), 'rect'] = peak_acceleration

duration = full_pw * 1.110
s = head_acceleration.Time.between(peak_time - (duration / 2), peak_time + (duration / 2))
pulse_time = head_acceleration.query(f"(Time >= 0) and (Time <= {duration})")['Time'].values
tri = peak_acceleration * (1 - abs(((2 * pulse_time) / duration)-1))
tri = list(tri)
tri = tri[:-1]
trunc_accel['tri'] = 0
trunc_accel.loc[s, 'tri'] = tri

duration = full_pw * 0.3169
s = head_acceleration.Time.between(peak_time - (duration / 2), peak_time + (duration / 2))
pulse_time = head_acceleration.query(f"(Time >= 0) and (Time <= {duration})")['Time'].values
quad = -4 * peak_acceleration / (duration**2) * (pulse_time - duration / 2)**2 + peak_acceleration
quad = list(quad)
print(f"duration: {duration}")
print(f"len(pulse_time): {len(pulse_time)}")
print(f"len(quad): {len(quad)}")
print(f"len space: {len(s)}")
quad = quad[:-1]
trunc_accel['quad'] = 0
trunc_accel.loc[s, 'quad'] = quad

duration = full_pw * 0.3484
s = head_acceleration.Time.between(peak_time - (duration / 2), peak_time + (duration / 2))
pulse_time = head_acceleration.query(f"(Time >= 0) and (Time <= {duration})")['Time'].values
sinefunc = peak_acceleration * np.sin(np.pi * pulse_time / duration)
sinefunc = list(sinefunc)
print(f"duration: {duration}")
print(f"len(pulse_time): {len(pulse_time)}")
print(f"len(sine): {len(sinefunc)}")
print(f"len space: {len(s)}")
trunc_accel['sinefunc'] = 0
trunc_accel.loc[s, 'sinefunc'] = sinefunc

duration = full_pw * 0.4875
s = head_acceleration.Time.between(peak_time - (duration / 2), peak_time + (duration / 2))
pulse_time = head_acceleration.query(f"(Time >= 0) and (Time <= {duration})")['Time'].values
hvsine = peak_acceleration * (np.sin(np.pi * pulse_time / duration))**2
hvsine = list(hvsine)
hvsine = hvsine[:-1]
trunc_accel['hvsine'] = 0
trunc_accel.loc[s, 'hvsine'] = hvsine

labels = ['Modeled Trace (Rectangular)', 'Modeled Trace (Triangular)', 'Modeled Trace (Quadratic)', 'Modeled Trace (Sine)', 'Modeled Trace (Haversine)']
curves = [trunc_accel.rect, trunc_accel.tri, trunc_accel.quad, trunc_accel.sinefunc, trunc_accel.hvsine]
for idx in [0, 1, 2, 3, 4]:
        fig = px.scatter(title='', width=width_inches * dpi, height=height_inches * dpi, template='simple_white')
        fig.add_scatter(x=head_acceleration['Time'], y=head_acceleration['Acceleration'], mode='lines', name='Experimental Trace', line=dict(color='#636EFA'))
        fig.update_layout(xaxis_title='Time (ms)',
                        yaxis_title='Resultant Head Acceleration (g)',
                        showlegend=True,
                        legend=dict(xanchor='right'),
                        font={'size': round(font_size * (dpi / 96)), 'family': "Times New Roman"},
                        xaxis_range=[0, 200], yaxis_range=[0, head_acceleration.Acceleration.max() * 1.2],
                        margin={'l': 20, 'r': 20, 't': 10, 'b': 20})

        fig.add_scatter(x=trunc_accel.Time, y=curves[idx], mode='lines', line=dict(dash='dash', color='black'), name=labels[idx])

        # legend

        fig.show()
        figure_name = f"{hics_folder}/{filename}_{idx}"
        fig.write_image(file=f"{figure_name}.png", format = "png", width=width_inches * dpi, height=height_inches * dpi, scale=1, engine='kaleido')
        os.startfile(f"{figure_name}.png")